In [1]:
import time
import datetime as dt
import hmac
import base64
import requests
from urllib.parse import urlencode

from pandas import json_normalize

In [3]:
APIKEY = ""
APISECRET = ""
PASS = ""
BASE_URL = 'https://www.okx.com'

In [4]:

def get_time():
    return dt.datetime.utcnow().isoformat()[:-3]+'Z'

def signature(timestamp, method, request_path, body, secret_key):
    if str(body) == '{}' or str(body) == 'None':
        body = ''
    message = str(timestamp) + str.upper(method) + request_path + str(body)
    mac = hmac.new(bytes(secret_key, encoding='utf8'), bytes(message, encoding='utf-8'), digestmod='sha256')
    d = mac.digest()
    return base64.b64encode(d)

# set request header
def get_header(request='GET', endpoint='', body:dict=dict()):
    cur_time = get_time()
    header = dict()
    header['CONTENT-TYPE'] = 'application/json'
    header['OK-ACCESS-KEY'] = APIKEY
    header['OK-ACCESS-SIGN'] = signature(cur_time, request, endpoint , body, APISECRET)
    header['OK-ACCESS-TIMESTAMP'] = str(cur_time)
    header['OK-ACCESS-PASSPHRASE'] = PASS
    return header

def send_signed_request(http_method, url_path, payload={}):
    '''
    See https://stackoverflow.com/questions/66486374/how-to-sign-an-okex-api-request
    '''

    url = BASE_URL + url_path
    header = get_header(http_method, url_path, payload)
    print(url)
    # print(header)
    response= requests.get(url, headers=header)
    response.json()
    return response.json()

# used for sending public data request
def send_public_request(url_path, payload={}):
    query_string = urlencode(payload, True)
    url = BASE_URL + url_path
    if query_string:
        url = url + '?' + query_string
    print("{}".format(url))
    response = requests.get(url)
    return response.json()

In [5]:
params = {
    'instType':'FUTURES'
}
response = send_public_request('/api/v5/market/tickers',params)
response = send_signed_request("GET", "/api/v5/account/balance", params)
# response
# json_normalize(response['data'])

https://www.okx.com/api/v5/market/tickers?instType=FUTURES
https://www.okx.com/api/v5/account/balance


In [6]:
def Get_instruments(instType = 'SWAP'):
    url = '/api/v5/public/instruments'
    params = {
        'instType':instType
    }
    response = send_public_request(url, params)
    response = json_normalize(response['data'])
    return response
    

In [7]:
def Get_funding_rate(instId = "BTC-USD-SWAP"):
    url = '/api/v5/public/funding-rate'
    params = {
        'instId':instId
    }
    response = send_public_request(url, params)
    # response = json_normalize(response['data'])
    response = response['data']
    return response

In [8]:
response = Get_funding_rate()
response

https://www.okx.com/api/v5/public/funding-rate?instId=BTC-USD-SWAP


[{'fundingRate': '-0.00016053',
  'fundingTime': '1643270400000',
  'instId': 'BTC-USD-SWAP',
  'instType': 'SWAP',
  'nextFundingRate': '-0.00035',
  'nextFundingTime': '1643299200000'}]

In [ ]:
def Get_funding_rate_history():

    url = '/api/v5/public/funding-rate-history'
    params = {
        'instId':instId
    }
    response = send_public_request(url, params)
    # response = json_normalize(response['data'])
    # response = response['data']
    
    return response